In [1]:
import json
import cv2 as cv
import cv2
import matplotlib.pyplot as plt
import urllib.request, urllib.error, urllib.parse
import numpy as np
import os
import bs4
from bs4 import *
from urllib.error import URLError, HTTPError
import urllib.request, urllib.parse, urllib.error
import glob
from functools import partial
import progressbar
import imageio
from multiprocessing import Pool
from PIL import Image
import pylab

from numba import cuda
import numba
import numpy as np
from functools import partial
import os


LOADING_BAR_LENGTH = 50
pos_training_path = 'Trainface'
neg_training_path = 'Trainnonface'
pos_testing_path = 'testface'
neg_testing_path = 'testnonface'
num_classifiers = 20
min_feature_height = 4
max_feature_height = 10
min_feature_width = 4
max_feature_width = 10

# Retrieving the images from url 
data=[];
with open('face_detection1.json') as json_data:
    c=54
    for line in json_data:
        c=c+1
        d = json.loads(line)
        url = json.dumps(d['content'])
        label = json.dumps(d['annotation'][0]['label'])
        url=url[1:-1]
        label=label[2:-2]
        print(url)
        print(label)
        try:
            urllib2.urlopen(url)
        except HTTPError as e1 :
            print(e1);
        except URLError as e2:
            print(e2);
        except ValueError as e3:
            print(e3)
        else:
            x=url.rfind(".jpg")
            y=url.rfind(".jpeg")
            z=url.rfind(".png")
            if x==-1:
                x=9999
            if y==-1:
                y=9999
            if z==-1:
                z=9999
            if y<=x and y <=z:
                temp=".jpeg"
            if x<=y and x <=z:
                temp=".jpg"
            if z<=x and z<=y:
                temp=".png"
            if label=="Face":
                urllib.urlretrieve(url,"Face/Image"+(str(c+1))+temp) 
            else:       
                urllib.urlretrieve(url,"NonFace/Image"+(str(c+1))+temp) 

In [2]:
def IntegralImage(img):
    
    integralImage=np.pad(img, 1, mode='constant')
    img=np.pad(img, 1, mode='constant')
    img=np.array(img)
    #print (img.shape)
    #print (img.shape[0])
    if ((img.shape[0])>(img.shape[1])):
        s=img.shape[1]-1
    else:
        s=img.shape[0]-1
    for m in range (1,s):
        i=m
        j=m
        for j in range(m,(img.shape[1]-1)):
            integralImage[i][j]=integralImage[i][j]+integralImage[i-1][j]+integralImage[i][j-1]-integralImage[i-1][j-1]
        j=m
        for i in range(m+1,(img.shape[0]-1)):
            integralImage[i][j]=integralImage[i][j]+integralImage[i-1][j]+integralImage[i][j-1]-integralImage[i-1][j-1]
    return  integralImage
def SumRegion(integralImage, topLeft, bottomRight):
    #img=np.array(integralImage)
    if topLeft == bottomRight:
        return integralImage[topLeft]
    topLeft=(topLeft[0],topLeft[1])
    topRight = (topLeft[0],bottomRight[1])
    bottomLeft = (bottomRight[0],topLeft[1])
    #print(type(integralImage[bottomRight[0]][bottomRight[1]]))
    s=integralImage[bottomRight[0]][bottomRight[1]] - integralImage[topRight[0]][topRight[1]] - integralImage[bottomLeft[0]][bottomLeft[1]] + integralImage[topLeft[0]][topLeft[1]]
    #print(type(s))
    return s

In [3]:
#### Testing
a=np.array([[1,2,3,4],[5,6,7,8]])
print (a)
print (IntegralImage(a))
b=IntegralImage(a)
print (SumRegion(b,[1,2],[2,4]))


[[1 2 3 4]
 [5 6 7 8]]
[[ 0  0  0  0  0  0]
 [ 0  1  3  6 10  0]
 [ 0  6 14 24 36  0]
 [ 0  0  0  0  0  0]]
15


In [4]:
def enum(**enums):
    return type('Enum', (), enums)
FeatureType = enum(TWO_VERTICAL=(1, 2), TWO_HORIZONTAL=(2, 1), THREE_HORIZONTAL=(3, 1), THREE_VERTICAL=(1, 3), FOUR=(2, 2))
FeatureTypes = [FeatureType.TWO_VERTICAL, FeatureType.TWO_HORIZONTAL, FeatureType.THREE_VERTICAL, FeatureType.THREE_HORIZONTAL, FeatureType.FOUR]

In [5]:
#import IntegralImage as ii
class HaarLikeFeature(object):
    #Class representing a haar-like feature.
    def __init__(self, feature_type, position, width, height, threshold, polarity):
        
        #Creates a new haar-like feature.
        #param feature_type: Type of new feature mentioned in FeatureType enum
        #type feature_type: violajonse.HaarLikeFeature.FeatureTypes
        #param position: Top left corner where the feature begins (x, y)
        #type position: (int, int)
        #param width: Width of the feature
        #type width: int
        #param height: Height of the feature
        #type height: int
        #param threshold: Feature threshold
        #type threshold: float
        #param polarity: polarity of the feature -1 or 1
        #type polarity: int
        self.type = feature_type
        self.topLeft = position
        self.bottomRight = (position[0] + width, position[1] + height)
        self.width = width
        self.height = height
        self.threshold = threshold
        self.polarity = polarity
        self.weight = 1
    def GetScore(self, integralImage):
        #Get score for given integral image array.
        #param integralImage: Integral image array
        #type integralImage: numpy.ndarray
        #return: Score for given feature
        #rtype: float
        score = 0
        if self.type == FeatureType.TWO_VERTICAL:
            first = SumRegion(integralImage,self.topLeft,(self.topLeft[0] + self.width,int(self.topLeft[1]+(self.height)/2)))
            second =SumRegion(integralImage,(self.topLeft[0],int(self.topLeft[1] + (self.height)/2)), self.bottomRight)
            score = first - second
           
        elif self.type == FeatureType.TWO_HORIZONTAL:
            first =  SumRegion(integralImage,self.topLeft,(int(self.topLeft[0]+self.width/2), self.topLeft[1] + self.height))
            second =SumRegion(integralImage,(int(self.topLeft[0] + self.width / 2), self.topLeft[1]), self.bottomRight)
            score = first - second
           
        elif self.type == FeatureType.THREE_HORIZONTAL:
            first =  SumRegion(integralImage, self.topLeft, (int(self.topLeft[0] + (self.width-1) / 3), self.topLeft[1] + self.height))
            second = SumRegion(integralImage, (int(self.topLeft[0] + self.width / 3),self.topLeft[1]),(int(self.topLeft[0] + 2 * self.width/3),self.topLeft[1] + self.height))
            third =  SumRegion(integralImage, (int(self.topLeft[0] + 2 * self.width / 3), self.topLeft[1]), self.bottomRight)
            score = first - second + third
            
        elif self.type == FeatureType.THREE_VERTICAL:
            first =  SumRegion(integralImage, self.topLeft, (self.bottomRight[0], int(self.topLeft[1] + self.height / 3)))
            second =SumRegion(integralImage, (self.topLeft[0], int(self.topLeft[1] + self.height / 3)), (self.bottomRight[0], int(self.topLeft[1] + 2 * self.height / 3)))
            third =  SumRegion(integralImage, (self.topLeft[0], int(self.topLeft[1] + 2 * self.height / 3)), self.bottomRight)
            score = first - second + third
           
        elif self.type == FeatureType.FOUR:
            # top left area
            first =  SumRegion(integralImage, self.topLeft, (int(self.topLeft[0] + self.width / 2), int(self.topLeft[1] + self.height / 2)))
            # top right area
            second = SumRegion(integralImage, (int(self.topLeft[0] + self.width / 2), self.topLeft[1]), (self.bottomRight[0], int(self.topLeft[1] + self.height / 2)))
            # bottom left area
            third =  SumRegion(integralImage, (self.topLeft[0], int(self.topLeft[1] + self.height / 2)), (int(self.topLeft[0] + self.width / 2), self.bottomRight[1]))
            # bottom right area
            fourth = SumRegion(integralImage, (int(self.topLeft[0] + self.width / 2), int(self.topLeft[1] + self.height / 2)), self.bottomRight)
            score = first - second - third + fourth
        #print (type(score))    
        return score
    
    def GetVote(self, integralImg):
        #Get vote of this feature for given integral image.
        #param integralImg: Integral image array
        #type int_img: numpy.ndarray
        #return: 1 iff this feature votes positively, otherwise -1
        #type: int
        score = self.GetScore(integralImg)
        #print (self.polarity)
        #print (self.threshold)
        #print (type(self.polarity))
        #print (type(self.threshold))
        #print (type(score))
        
        try:    
            if score*self.polarity < (self.polarity * self.threshold):
                vote=1
            else:
                vote=-1
        except:
            #print(self.type)
            
            return 0
        #print('Y')
        return self.weight * (vote)

In [6]:
def _get_feature_vote(feature, image):
    return feature.GetVote(image)
def _create_features(img_height, img_width, min_feature_width, max_feature_width, min_feature_height, max_feature_height):
    print('Creating haar-like features..')
    features = []
    for feature in FeatureTypes:
        # FeatureTypes are just tuples
        feature_start_width = max(min_feature_width, feature[0])
        for feature_width in range(feature_start_width, max_feature_width, feature[0]):
            feature_start_height = max(min_feature_height, feature[1])
            for feature_height in range(feature_start_height, max_feature_height, feature[1]):
                for x in range(img_width - feature_width):
                    for y in range(img_height - feature_height):
                        features.append(HaarLikeFeature(feature, (x, y), feature_width, feature_height, 0, 1))
                        features.append(HaarLikeFeature(feature, (x, y), feature_width, feature_height, 0, -1))
    print('..done. ' + str(len(features)) + ' features created.\n')
    return features

In [7]:
def learn(positive_iis, negative_iis, num_classifiers=-1, min_feature_width=1, max_feature_width=-1, min_feature_height=1, max_feature_height=-1):
    #Selects a set of classifiers. Iteratively takes the best classifiers based
    #on a weighted error.
    #param positive_iis: List of positive integral image examples
    #type positive_iis: list[numpy.ndarray]
    #param negative_iis: List of negative integral image examples
    #type negative_iis: list[numpy.ndarray]
    #param num_classifiers: Number of classifiers to select, -1 will use all
    #classifiers
    #type num_classifiers: int
    #return: List of selected features
    #rtype: list[violajones.HaarLikeFeature.HaarLikeFeature]
    num_pos = len(positive_iis)
    num_neg = len(negative_iis)
    num_imgs = num_pos + num_neg
    img_height, img_width = positive_iis[0].shape
    # Maximum feature width and height default to image width and height
    max_feature_height = img_height #if max_feature_height == -1 else max_feature_height
    max_feature_width = img_width #if max_feature_width == -1 else max_feature_width
    # Create initial weights and labels
    pos_weights = np.ones(num_pos) * 1. / (2 * num_pos)
    neg_weights = np.ones(num_neg) * 1. / (2 * num_neg)
    weights = np.hstack((pos_weights, neg_weights))
    labels = np.hstack((np.ones(num_pos), np.ones(num_neg) * -1))
    images = positive_iis + negative_iis
    # Create features for all sizes and locations
    features = _create_features(img_height, img_width, min_feature_width, max_feature_width, min_feature_height, max_feature_height)
    num_features = len(features)
    feature_indexes = list(range(num_features))

    num_classifiers = num_features if num_classifiers == -1 else num_classifiers

    print('Calculating scores for images..')
    votes =((np.zeros((num_imgs, num_features))))
    print (features[0].weight)
    #k=0
    for i in range(0,num_imgs):
        for j in range(0,num_features):
            
            votes[i][j] = _get_feature_vote(features[j],images[i])
            #print votes
    #print(votes)
    # select classifiers
    classifiers = []
    print('Selecting classifiers..')
    bar = progressbar.ProgressBar()
    for _ in bar(list(range(num_classifiers))):
        classification_errors = np.zeros(len(feature_indexes))
        weights *= 1. / np.sum(weights)
        # select best classifier based on the weighted error
        for f in range(len(feature_indexes)):
            f_idx = feature_indexes[f]
            # classifier error is the sum of image weights where the classifier
            # is right
            error = sum([weights[img_idx] if labels[img_idx] != votes[img_idx, f_idx] else 0 for img_idx in range(num_imgs)])
            classification_errors[f] = error

        # get best feature, i.e. with smallest error
        min_error_idx = np.argmin(classification_errors)
        best_error = classification_errors[min_error_idx]
        best_feature_idx = feature_indexes[min_error_idx]

        # set feature weight
        best_feature = features[best_feature_idx]
        feature_weight = 0.5 * np.log((1 - best_error) / best_error)
        best_feature.weight = feature_weight

        classifiers.append(best_feature)

        # update image weights
        weights = np.array(list([weights[img_idx] * np.sqrt((1-best_error)/best_error) if labels[img_idx] != votes[img_idx, best_feature_idx] else weights[img_idx] * np.sqrt(best_error/(1-best_error)) for img_idx in range(num_imgs)]))

        # remove feature (a feature can't be selected twice)
        feature_indexes.remove(best_feature_idx)
    return classifiers,votes


In [8]:
faces_training=[]
faces_ii_training=[]
non_faces_training=[]
non_faces_ii_training=[]

print('Loading faces for training..')
for file in glob.glob('Trainface/*'):
    try:
        #im = imageio.imread(file)
        im = imageio.imread(file)
        im=cv2.resize(im,(19,19))
        try:
            im= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        except:
            print('Unexpected Error')
        faces_training.append(im)
        faces_ii_training.append(IntegralImage(im))
    except IndexError as e:
        e
        #print (e)
        
print('..done. ' + str(len(faces_training)) + ' faces loaded.\n\nLoading non faces..')
print (len(faces_training))
print (len(faces_ii_training))
for file in glob.glob('Trainnonface/*'):
    try:    
        im = imageio.imread(file)
        im=cv2.resize(im,(19,19))
        try:
            im= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        except:
            print('Unexpected Error')
        non_faces_training.append(im)
        non_faces_ii_training.append(IntegralImage(im))
    except IndexError as e:
        print (e)
print('..done. ' + str(len(non_faces_training)) + ' non faces loaded.\n')



Loading faces for training..


/home/vclab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in ubyte_scalars
  app.launch_new_instance()
/home/vclab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in ubyte_scalars


Unexpected Error
..done. 100 faces loaded.

Loading non faces..
100
100
..done. 113 non faces loaded.



In [9]:
classifiers,votes = learn(faces_ii_training, non_faces_ii_training, num_classifiers, min_feature_height, max_feature_height, min_feature_width, max_feature_width)


Creating haar-like features..
..done. 97578 features created.

Calculating scores for images..
1


/home/vclab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in ubyte_scalars
/home/vclab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: overflow encountered in ubyte_scalars
/home/vclab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: overflow encountered in ubyte_scalars
/home/vclab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: overflow encountered in ubyte_scalars
/home/vclab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: overflow encountered in ubyte_scalars
/home/vclab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: overflow encountered in ubyte_scalars
N/A% (0 of 20) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Selecting classifiers..


100% (20 of 20) |########################| Elapsed Time: 0:01:42 Time:  0:01:42


In [ ]:
for i in range(0,votes.shape[0]):
    for j in range(0,votes.shape[1]):
        if votes[i][j]==1:
            print(1)

In [ ]:
def ensemble_vote(int_img, classifiers):
    return 1 if sum([c.GetVote(int_img) for c in classifiers]) >= 0 else 0

In [ ]:
def ensemble_vote_all(int_imgs, classifiers):
    vote_partial = partial(ensemble_vote, classifiers=classifiers)
    return list(map(vote_partial, int_imgs))


In [ ]:
faces_test=[]
faces_ii_test=[]
non_faces_test=[]
non_faces_ii_test=[]

print('Loading faces for testing..')
for file in glob.glob('testface/*'):
    try:
        #im = imageio.imread(file)
        im = imageio.imread(file)
        im=cv2.resize(im,(100,100))
        try:
            im= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        except:
            print('Unexpected Error')
        faces_test.append(im)
        faces_ii_test.append(IntegralImage(im))
    except IndexError as e:
        e
        #print (e)
print('..done. ' + str(len(faces_test)) + ' faces loaded.\n\nLoading non faces..')

for file in glob.glob('testnonface/*'):
    try:    
        im = imageio.imread(file)
        im=cv2.resize(im,(100,100))
        try:
            im= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        except:
            print('Unexpected Error')
        non_faces_test.append(im)
        non_faces_ii_test.append(IntegralImage(im))
    except IndexError as e:
        print (e)
print('..done. ' + str(len(non_faces_test)) + ' non faces loaded.\n')


In [ ]:
print('Testing selected classifiers..')
correct_faces = 0
correct_non_faces = 0
correct_faces = sum(ensemble_vote_all(faces_ii_test, classifiers))
correct_non_faces = len(non_faces_test) - sum(ensemble_vote_all(non_faces_ii_test, classifiers))

print('..done.\n\nResult:\n      Faces: ' + str(correct_faces) + '/' + str(len(faces_test))
          + '  (' + str((float(correct_faces) / len(faces_test)) * 100) + '%)\n  non-Faces: '
          + str(correct_non_faces) + '/' + str(len(non_faces_test)) + '  ('
          + str((float(correct_non_faces) / len(non_faces_test)) * 100) + '%)')

    